In [ ]:
#todos:
#1) check how it handles more than one calypso at a time
#2)

In [1]:
from __future__ import print_function
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import RDD
from pyspark.sql.window import Window
from pyspark.sql.utils import require_minimum_pandas_version, require_minimum_pyarrow_version
import pandas as pd
require_minimum_pandas_version()
require_minimum_pyarrow_version()
from pyspark.sql import HiveContext
from pyspark.sql import SQLContext as sqlContext
from pyspark.sql.functions import col, split, when, row_number, lit, regexp_replace
from itertools import chain
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType
import numpy as np

In [2]:
##2## Create the Data warehouse directory

data_lake = "hdfs://namenode:8020/data_hub/data/lake"

# Enable Arrow-based columnar data transfers
# Set warehouse.dir explicitly : https://stackoverflow.com/a/38945867/4032515
spark = SparkSession \
        .builder \
        .master("spark://spark-master:7077") \
        .appName("quote data explorer") \
        .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
        .config("spark.sql.execution.arrow.enabled", "false") \
        .config("spark.sql.shuffle.partitions", "10") \
        .config("spark.cores.max", "4") \
        .config("spark.executor.cores", "4") \
        .enableHiveSupport() \
        .getOrCreate()

spark.catalog.setCurrentDatabase("client_pohyola_dept1_data_spec_test_4")

hc = HiveContext(spark.sparkContext)

In [3]:
conv_rim = "client_pohyola_dept1_data_spec_test_4.ds_0001"
conv_rim_1 = spark.table(conv_cal).where(col("io_yields_convert_corr_id") == 51)
conv_rim = [conv_rim_1]
#conv_cal = "client_pohyola_dept1_data_spec_reduced_2.ds_0002"
#conv_cal_1 = spark.table(conv_cal).where(col("io_yields_convert_corr_id") == 9)
#conv_cal_2 = spark.table(conv_cal).where(col("io_yields_convert_corr_id") == 10)
#conv_cal_3 = spark.table(conv_cal).where(col("io_yields_convert_corr_id") == 11)
#currently works for one partition:
#conv_cal = [conv_cal_1]

In [4]:
####-Function 1: RIM only

def rim_function(rim, preexistingTrait):
    if preexistingTrait is None:

        rim_initial_df = spark.table(rim) 
#                .where(col("io_yields_convert_corr_id") == 8) \
#                .withColumnRenamed('Sopno', 'LoanReference') \
#                .withColumn('LoanID', col("AlkuPerainenSopimusNumeroID")) \
#                .withColumnRenamed('AlkuPerainenSopimusNumeroID', 'OriginalContractId') \
#                .withColumnRenamed('SopimusnumeroId', 'CurrentContractId') \
#                .withColumnRenamed('EnsimmainenAvausPvmDt', 'OriginalOpeningDt') \
#                .withColumnRenamed('LyhennysMaksutapa', 'AmortizationTypeLJR') \
#                .withColumnRenamed('LyhennysMaksufrekvenssi', 'AmortizationFrequencyLJR') \
#                .withColumnRenamed('VastuuTaseessaAmt', 'CurrentNotionalAmt') \
#                .withColumnRenamed('MyonnettyAmt', 'OriginalNotionalAmt') \
#                .withColumnRenamed('PaattymisPvmDt', 'LoanMaturityDt') \
#                .withColumnRenamed('LyhennysRahamaara', 'AmortizationAmt') \
#                .withColumnRenamed('MarginaaliVelkaPros', 'MarginalRatePercent') \
#                .drop("io_yields_convert_corr_id") \
#                .drop("io_yields_record_id")

        print("This is the schema of RIM: ") 
        rim_initial_df.printSchema()
        #rim_initial_df.select("LoanReference","OriginalContractId","LoanID").show(100)
        print("Number of rows of initial rim: " + str(rim_initial_df.count()))
        return rim_initial_df.toPandas()
    else:
        print("todo by client")

In [5]:
####-Function 2: Calypso

def calypso_function(calypsoPartitions, rim, preexistingTrait):
    #sc = spark.sparkContext
    #dff = spark.createDataFrame(sc.emptyRDD(), StructType([]))
    
    if preexistingTrait is None:
        
        for idx,x in enumerate(calypsoPartitions):
            df = calypso_aux(x)
            calypso_references = df['LoanReference'].values
            #print(calypso_references[0])
            
            if idx == 0:
                dff = df
                if len(calypsoPartitions) > 1:
                    dfi = df
            #Standalone calypso
            if rim is None:
                
                dff = dff.drop("LoanID", 1)
                dff["LoanID"] = dff["CurrentContractId"]
                break
            #Rim/calypso merge
            else:
                
                df_rim = rim_function(rim, None)
                rim_references = df_rim['LoanReference'].values
                #print(rim_references)
                
                if any(ref in calypso_references for ref in rim_references):
                    #print("it exists!")
                    existingReferences = set(rim_references).intersection(calypso_references)
                    #print(existingReferences)
                    
                    for y in existingReferences:
                        #print(y)
                        selectedID = df_rim["OriginalContractId"][df_rim["LoanReference"] == str(y)].unique()
                        #print(selectedID[0])
                        df.loc[df['LoanReference'] == str(y), 'LoanID'] = selectedID[0]
                        df.loc[df['LoanReference'] != str(y), 'LoanID'] = df['CurrentContractId']
                        
                    if idx ==0:
                        dff = pd.concat([df_rim, df], ignore_index=True)
                    else:
                        dfi = pd.concat([dff, df], ignore_index=True)
                else:
                    #Standalone calypso
                    dff = dff.drop("LoanID", 1)
                    dff["LoanID"] = dff["CurrentContractId"]
                    if idx ==0:
                        dff = pd.concat([df_rim, df], ignore_index=True)
                    else:    
                        dfi = pd.concat([dff, df], ignore_index=True)
    else:
        print("todo by client")
    
    if len(calypsoPartitions) > 1:
        dff = concat([dff, dfi], ignore_index=True)
    return dff

        
#Auxiliary function to read each time a calypso increment
def calypso_aux(oneCalypso):

    calypso_df = oneCalypso \
        .withColumnRenamed('LoanReferenceNumber', 'LoanReference') \
        .withColumnRenamed('CalypsotradeID', 'CurrentContractId') \
        .withColumnRenamed('LoanStartDate', 'OriginalOpeningDt') \
        .withColumnRenamed('AmortizationType', 'AmortizationTypeLJR') \
        .withColumnRenamed('AmortizationFrequency', 'AmortizationFrequencyLJR') \
        .withColumnRenamed('AmortizationAmount', 'AmortizationAmt') \
        .withColumnRenamed('currentnotional', 'CurrentNotionalAmt') \
        .withColumnRenamed('originalnotional', 'OriginalNotionalAmt') \
        .withColumnRenamed('LoanMaturitydate', 'LoanMaturityDt') \
        .withColumnRenamed('totalmarginal', 'MarginalRatePercent') \
        .withColumn("LoanID", lit(None)) \
        .withColumn("OriginalContractId", lit(None)) \
        .drop("io_yields_convert_corr_id") \
        .drop("io_yields_record_id")
        
    print("This is the schema of calypso: ") 
    calypso_df.printSchema()
    #print("Number of rows of calypso: " + str(calypso_df.count()))
    return calypso_df.toPandas()

In [6]:
#### Run functions

#result = rim_function(conv_rim, None)
#result = calypso_function(conv_cal, None, None)
result = calypso_function(conv_cal, conv_rim, None)

print("This is the schema of final df: ")
#--------------
# result.printSchema()
# print("Number of rows of final df: " + str(result.count()))
print(list(result))
print("Number of rows of final df: " + str(result.shape[0]))
print(result[['Pvm_date','LoanReference', 'LoanID', 'CurrentContractId']])
#--------------

This is the schema of calypso: 
root
 |-- Pvm_date: string (nullable = true)
 |-- LoanReference: string (nullable = true)
 |-- CurrentContractId: integer (nullable = true)
 |-- LoanMaturityDt: string (nullable = true)
 |-- OriginalOpeningDt: string (nullable = true)
 |-- OriginalNotionalAmt: string (nullable = true)
 |-- CurrentNotionalAmt: string (nullable = true)
 |-- AmortizationTypeLJR: integer (nullable = true)
 |-- AmortizationFrequencyLJR: integer (nullable = true)
 |-- AmortizationAmt: double (nullable = true)
 |-- MarginalRatePercent: double (nullable = true)
 |-- LoanID: null (nullable = true)
 |-- OriginalContractId: null (nullable = true)

This is the schema of RIM: 
root
 |-- Pvm_date: string (nullable = true)
 |-- LoanReference: string (nullable = true)
 |-- OriginalContractId: integer (nullable = true)
 |-- CurrentContractId: integer (nullable = true)
 |-- LoanMaturityDt: string (nullable = true)
 |-- OriginalOpeningDt: string (nullable = true)
 |-- OriginalNotionalAmt: 

In [7]:
result_2 = result.dropna()

result_2["Pvm_date"] = pd.to_datetime(result_2["Pvm_date"], format = '%m/%d/%Y').values.astype(np.int64) // 10 ** 9
result_2["OriginalOpeningDt"] = pd.to_datetime(result_2["OriginalOpeningDt"], format = '%m/%d/%Y').values.astype(np.int64) // 10 ** 9
result_2["LoanMaturityDt"] = pd.to_datetime(result_2["LoanMaturityDt"], format = '%m/%d/%Y').values.astype(np.int64) // 10 ** 9

result_2["LoanReference"] = result_2["LoanReference"].replace(',?','', regex=True).astype(float)
result_2["CurrentNotionalAmt"] = result_2["CurrentNotionalAmt"].replace(',?','', regex=True).astype(float)
result_2["OriginalNotionalAmt"] = result_2["OriginalNotionalAmt"].replace(',?','', regex=True).astype(float)
result_2["AmortizationAmt"] = result_2["AmortizationAmt"].replace(',?','', regex=True).astype(float)

label = result_2["LoanMaturityDt"]
result_2 = result_2.drop(columns = ["LoanMaturityDt"])
nb_features = result_2.shape[1]
mat = result_2.as_matrix()
label = label.as_matrix()

/opt/conda/envs/yields-dev-ds/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/envs/yields-dev-ds/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/envs/yields-dev-ds/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [9]:
for i in range(nb_features):
    max_f, min_f = np.max(mat[:, i]), np.min(mat[:, i])
    mat[:, i] = (mat[:, i] - min_f) / (max_f - min_f)

label = (label - np.min(label))/(np.max(label) - np.min(label))

In [30]:
import tensorflow as tf

nb_features = mat.shape[1]
nb_obs = mat.shape[0]

label = label.reshape([nb_obs, 1])
mat = mat.reshape([nb_obs, nb_features])

x = tf.placeholder("float", [nb_obs, nb_features])
y = tf.placeholder("float", [nb_obs, 1])

weight = tf.Variable(tf.random_normal([nb_features, 1]), name = 'weights')
bias = tf.Variable(tf.random_normal([1]), name = 'bias')

pred = tf.add(tf.matmul(x, weight), bias)

loss = tf.reduce_mean(tf.squared_difference(pred, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(loss)

init = tf.global_variables_initializer()

In [42]:
LOG_DIR = "tb"

sess = tf.InteractiveSession()
sess.run(init)
for i in range(500):
    sess.run(optimizer, feed_dict = {x: mat, y: label})
    loss_value, pred_value = sess.run([loss, pred], feed_dict = {x: mat, y: label})
    if i%100 == 0:
        print("Epoch", i)
        print("Error:", loss_value)

from tensorflow.contrib.tensorboard.plugins import projector
import os

# Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = x.name
# Link this tensor to its metadata file (e.g. labels).

# Use the same LOG_DIR where you stored your checkpoint.
summary_writer = tf.summary.FileWriter(LOG_DIR)

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)
        
saver = tf.train.Saver()
saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), 1)

sess.close()

Epoch 0
Error: 1.1257684
Epoch 100
Error: 0.1399353
Epoch 200
Error: 0.065179504
Epoch 300
Error: 0.043342214
Epoch 400
Error: 0.034730785
